# Обучение нейрона с помощью функции потерь LogLoss

<h3 style="text-align: center;"><b>Нейрон с сигмоидой</b></h3>

Снова рассмотрим нейрон с сигмоидой, то есть $$f(x) = \sigma(x)=\frac{1}{1+e^{-x}}$$ 

Ранее мы установили, что **обучение нейрона с сигмоидой с квадратичной функцией потерь**:  

$$MSE(w, x) = \frac{1}{2n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{2n}\sum_{i=1}^{n} (\sigma(w \cdot x_i) - y_i)^2$$    

где $w \cdot x_i$ - скалярное произведение, а $\sigma(w \cdot x_i) =\frac{1}{1+e^{-w \cdot x_i}} $ - сигмоида -- **неэффективно**, то есть мы увидели, что даже за большое количество итераций нейрон предсказывает плохо.

Давайте ещё раз взглянем на формулу для градиентного спуска от функции потерь $MSE$ по весам нейрона:

$$ \frac{\partial MSE}{\partial w} = \frac{1}{n} X^T (\sigma(w \cdot X) - y)\sigma(w \cdot X)(1 - \sigma(w \cdot X))$$

А теперь смотрим на график сигмоиды:

<img src="https://cdn-images-1.medium.com/max/1200/1*IDAnCFoeXqWL7F4u9MJMtA.png" width=500px height=350px>

**Её значения: числа от 0 до 1.**

Если получше проанализировать формулу, то теперь можно заметить, что, поскольку сигмоида принимает значения между 0 и 1 (а значит (1-$\sigma$) тоже принимает значения от 0 до 1), то мы умножаем $X^T$ на столбец $(\sigma(w \cdot X) - y)$ из чисел от -1 до 1, а потом ещё на столбцы $\sigma(w \cdot X)$ и $(1 - \sigma(w \cdot X))$ из чисел от 0 до 1. Таким образом в лучшем случае $\frac{\partial{Loss}}{\partial{w}}$ будет столбцом из чисел, порядок которых максимум 0.01 (в среднем, понятно, что если сигмоида выдаёт все 0, то будет 0, если все 1, то тоже 0). После этого мы умножаем на шаг градиентного спуска, который обычно порядка 0.001 или 0.1 максимум. То есть мы вычитаем из весов числа порядка ~0.0001. Медленновато спускаемся, не правда ли? Это называют **проблемой затухающих градиентов**.

Чтобы избежать эту проблему в задачах классификации, в которых моделью является нейрон с сигмоидной функцией активации, предсказывающий "вероятности" принадлженостей к классамиспользуют **LogLoss**:  

$$J(\hat{y}, y) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\sigma(w \cdot x_i)) + (1 - y_i) \log(1 - \sigma(w \cdot x_i))$$

где, как и прежде, $y$ - столбец $(n, 1)$ из истинных значений классов, а $\hat{y}$ - столбец $(n, 1)$ из предсказаний нейрона.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

In [ ]:
def loss(y_pred, y):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

Отметим, что сейчас речь идёт именно о **бинарной классификации (на два класса)**, в многоклассовой классификации используется функция потерь под названием *кросс-энтропия*, которая является обобщением LogLoss'а на случай нескольких классов.

Почему же теперь всё будет лучше? Раньше была проблема умножения маленьких чисел в градиенте. Давайте посмотрим, что теперь:

* Для веса $w_j$:

$$ \frac{\partial Loss}{\partial w_j} = 
-\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i)}\right)(\sigma(w \cdot x_i))_{w_j}' = -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i)}\right)\sigma(w \cdot x_i)(1 - \sigma(w \cdot x_i))x_{ij} = $$
$$-\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{ij}$$

* Градиент $Loss$'а по вектору весов -- это вектор, $j$-ая компонента которого равна $\frac{\partial Loss}{\partial w_j}$ (помним, что весов всего $m$):

$$\begin{align}
    \frac{\partial Loss}{\partial w} &= \begin{bmatrix}
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i1} \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i2} \\
           \vdots \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{im}
         \end{bmatrix} 
\end{align}=\frac{1}{n} X^T \left(\hat{y} - y\right)$$ 

По аналогии с $w_j$ выведите формулу для свободного члена (bias'а) $b$ (*hint*: можно считать, что при нём есть признак $x_{i0}=1$ на всех $i$):

$$\begin{align}
    \frac{\partial Loss}{\partial b} &= \begin{bmatrix}
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i + b)\right)
         \end{bmatrix} 
\end{align}=\frac{1}{n}  \left(\hat{y} - y\right)$$ 

Получили новое правило для обновления $w$ и $b$. 

In [ ]:
def sigmoid(x):
    """Сигмоидальная функция"""
    x = x.astype('float')
    return 1 / (1 + np.exp(-x))

Реализуйте нейрон с функцией потерь LogLoss:

In [ ]:
class Neuron:
    
    def __init__(self, w=None, b=0):
        """
        :param: w -- вектор весов
        :param: b -- смещение
        """
        # пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
        
        
    def activate(self, x):
        return sigmoid(x)
    
        
    def forward_pass(self, X):
        """
        Эта функция рассчитывает ответ нейрона при предъявлении набора объектов
        :param: X -- матрица объектов размера (n, m), каждая строка - отдельный объект
        :return: вектор размера (n, 1) из нулей и единиц с ответами перцептрона 
        """
        a = self.activate(np.dot(X,self.w) + self.b)
        return a
        
    
    def backward_pass(self, X, y, y_pred, learning_rate=0.1):
        """
        Обновляет значения весов нейрона в соответствие с этим объектом
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                learning_rate - "скорость обучения" (символ alpha в формулах выше)
        В этом методе ничего возвращать не нужно, только правильно поменять веса
        с помощью градиентного спуска.
        """
        # тут нужно обновить веса по формулам, написанным выше
        grad_w = (1 / X.shape[0]) * X.T @ (y_pred - y)
        grad_b = (1 / X.shape[0]) * (y_pred-y).sum()
        self.w -=  learning_rate * grad_w
        self.b  -= learning_rate * grad_b
    
    def fit(self, X, y, num_epochs=5000):
        """
        Спускаемся в минимум
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                num_epochs -- количество итераций обучения
        :return: J_values -- вектор значений функции потерь
        """
        self.w = np.zeros((X.shape[1], 1))  # столбец (m, 1)
        self.b = 0  # смещение
        loss_values = []  # значения функции потерь на различных итерациях обновления весов
        y = y.reshape(y.shape[0],1)
        
        for i in range(num_epochs):
            # предсказания с текущими весами
            y_pred = self.forward_pass(X)
            # считаем функцию потерь с текущими весами
            loss_values.append(loss(y_pred, y))
            # обновляем веса по формуле градиентного спуска
            self.backward_pass(X, y, y_pred)
        return loss_values

<h3 style="text-align: center;"><b>Тестирование</b></h3>

Протестируем нейрон, обученный с новой функцией потерь, на тех же данных, что и в предыдущем ноутбуке:

**Проверка forward_pass()**

In [ ]:
w = np.array([1., 2.]).reshape(2, 1)
b = 2.
X = np.array([[1., 3.],
              [2., 4.],
              [-1., -3.2]])

neuron = Neuron(w, b)
y_pred = neuron.forward_pass(X)
print ("y_pred = " + str(y_pred))

y_pred = [[0.99987661]
 [0.99999386]
 [0.00449627]]


**Проверка backward_pass()**

In [ ]:
y = np.array([1, 0, 1]).reshape(3, 1)

In [ ]:
neuron.backward_pass(X, y, y_pred)

print ("w = " + str(neuron.w))
print ("b = " + str(neuron.b))

w = [[0.9001544 ]
 [1.76049276]]
b = 1.9998544421863216


Проверьте на наборах данных "яблоки и груши" и "голос".

In [ ]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1rvnE6CKEySAGCvGct-MeSdAaLu4CujeJ"}) 
downloaded.GetContentFile('applespears.csv')

In [ ]:
data_apple = pd.read_csv('applespears.csv')

In [ ]:
X = data_apple.drop(columns = ['target'])
y = data_apple['target']
y = y.to_numpy()
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [ ]:
neuron = Neuron()

In [ ]:
neuron.fit(X_train,y_train)

[0.6931471805599454,
 0.6896577965087032,
 0.6863207194248128,
 0.6831249725628519,
 0.6800603859499561,
 0.6771175415291837,
 0.6742877211264968,
 0.6715628572713147,
 0.6689354868584353,
 0.6663987076059628,
 0.6639461372383025,
 0.661571875304117,
 0.659270467525259,
 0.6570368725632084,
 0.6548664310836092,
 0.6527548369964564,
 0.6506981107487243,
 0.6486925745472705,
 0.6467348293922812,
 0.6448217338049985,
 0.6429503841377091,
 0.6411180963587543,
 0.6393223892104353,
 0.6375609686430209,
 0.6358317134334575,
 0.63413266190278,
 0.6324619996515254,
 0.6308180482376254,
 0.6291992547262442,
 0.6276041820458189,
 0.6260315000891296,
 0.6244799775025485,
 0.6229484741107209,
 0.6214359339277715,
 0.6199413787097545,
 0.6184639020064366,
 0.6170026636736732,
 0.6155568848105692,
 0.6141258430883625,
 0.6127088684405031,
 0.6113053390857683,
 0.609914677858429,
 0.6085363488215131,
 0.6071698541410773,
 0.6058147312011295,
 0.6044705499404365,
 0.6031369103939276,
 0.601813440422761

In [ ]:
predicted = neuron.forward_pass(X_test)

In [ ]:
predicted[predicted >= 0.5] = 1
predicted[predicted  < 0.5] = 0 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test.astype(float),predicted)

0.97

In [ ]:
downloaded = drive.CreateFile({'id':"1P0XApm2tPrJVcRIlOPtSckN0QOcCavbg"}) 
downloaded.GetContentFile('voice.csv')

In [ ]:
voice_data = pd.read_csv('voice.csv')

In [ ]:
X_data = voice_data.drop(columns= ['label'])
X_data = (X_data - X_data.min()) / (X_data.max() - X_data.min())
y = voice_data['label']
y = y.to_numpy()
y[y == 'male'] = 1
y[y == 'female'] = 0
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.3)

In [ ]:
neuron = Neuron()
neuron.fit(X_train,y_train)

[0.6931471805599092,
 0.6912776715306285,
 0.6894273363275487,
 0.6875956877374717,
 0.6857823147891255,
 0.6839868625931296,
 0.6822090176531631,
 0.6804484971650686,
 0.6787050412220932,
 0.6769784071376741,
 0.6752683653107582,
 0.6735746962146996,
 0.6718971882043271,
 0.670235635918746,
 0.6685898391177456,
 0.6669596018337689,
 0.6653447317534706,
 0.6637450397661888,
 0.6621603396337815,
 0.6605904477486326,
 0.6590351829556088,
 0.657494366420452,
 0.6559678215317605,
 0.6544553738272716,
 0.6529568509376641,
 0.651472082543,
 0.6500009003381569,
 0.6485431380047425,
 0.647098631187546,
 0.645667217474188,
 0.6442487363769958,
 0.6428430293163729,
 0.6414499396051678,
 0.6400693124336462,
 0.6387009948548766,
 0.6373448357702458,
 0.6360006859150764,
 0.6346683978441908,
 0.6333478259173791,
 0.6320388262847677,
 0.6307412568720093,
 0.6294549773653327,
 0.6281798491964314,
 0.6269157355271593,
 0.6256625012341173,
 0.624420012893055,
 0.6231881387631443,
 0.6219667487711407,
 

In [ ]:
pred = neuron.forward_pass(X_test)

In [ ]:
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0

In [ ]:
accuracy_score(y_test.astype('float'),pred)

0.9589905362776026